In [5]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'

from mpl_toolkits.basemap import Basemap
from mpl_toolkits.axes_grid1 import make_axes_locatable
from pyproj import Proj
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr
import datetime

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax,lat_0,lon_0,ny,nx):

    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
#     m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='tmerc', ax=ax,lat_0=lat_0,lon_0=lon_0)

    m.drawstates(linewidth=0.5, linestyle='solid', color='grey')
    m.drawcountries(linewidth=0.5, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.25, linestyle='solid', color='k')
    return m

def create_colormap(cmap):
    # create a white-cmap linear colormap
    # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
    cmap = mpl.cm.get_cmap(cmap) #‘viridis’，'jet' # get the blue color of jet 
    c0 = cmap(0.0)
#     top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])
    top = mpl.colors.LinearSegmentedColormap.from_list("", ["white","white"])

    # combine two liner colormaps to create a
    # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
    bottom = mpl.cm.get_cmap(cmap) #'jet','viridis'
    newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.01))),bottom(np.linspace(0, 1, int(256*0.99)))))
    newcmp = mpl.colors.LinearSegmentedColormap.from_list("NewCmap", newcolors)
    return newcmp

# set the colormap and centre the colorbar
class MidpointNormalize(mpl.colors.Normalize):
    """Normalise the colorbar.
    source: http://chris35wills.github.io/matplotlib_diverging_colorbar/
    e.g. im=ax1.imshow(array, norm=MidpointNormalize(midpoint=0.,vmin=-300, vmax=1000))    
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        mpl.colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')

gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')
stn_regr_dir = '/glade/u/home/hongli/scratch/2020_08_17stn_gmet/stn_gmet/gmet_regr_summary'
nldas_regr_dir = os.path.join(root_dir,'test_uniform_perturb')

stn_ens_std_file = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet/data/stn_ens_summary/ens_forc.sumamry.2016.nc'

test_folders = [d for d in os.listdir(nldas_regr_dir)]
test_folders = sorted(test_folders)

regress_subforlder = 'gmet_regr'
ens_subforlder = 'gmet_ens_bc'

ens_num = 100
time_format = '%Y-%m-%d'

dpi_value = 150
# plot_date_start = '2012-01-01' #'1979-01-01' #'2013-01-01'
# plot_date_end = '2016-12-31' #'2019-12-31' #'2016-12-31'
plot_date_start = '2016-06-01' #'1979-01-01' #'2013-01-01'
plot_date_end = '2016-06-30' #'2019-12-31' #'2016-12-31'

plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step36_plot_spatial_precip')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
#======================================================================================================
print('Read gridinfo mask')
# get xy mask from gridinfo.nc
f_gridinfo = xr.open_dataset(gridinfo_file)
mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
#data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
latitude = f_gridinfo['latitude'].values[:]
longitude = f_gridinfo['longitude'].values[:]

#======================================================================================================
# read station regression results
print('Read station ens std')
output_basename = os.path.join(stn_regr_dir, 'regress_ts_')

print(' -- read spatial uncertainty')
f=xr.open_dataset(stn_ens_std_file)
time = f['time'].values[:]
stn_time_regr = pd.to_datetime(time)

stn_pcp_std = f['pcp_std'].values[:]
stn_tmean_std = f['tmean_std'].values[:]
stn_trange_std = f['trange_std'].values[:]

# define plot mask for stn regr
mask_stn_t = (stn_time_regr>=plot_date_start_obj) & (stn_time_regr<=plot_date_end_obj)
mask_stn_xy = (np.isnan(stn_tmean_std[0,:,:]))
mask_both = (mask_stn_xy==True) | (mask_xy==0)

print(' -- calculate temporal mean')
# caluclate time series mean(ny,nx)
stn_pcp_std_mean = np.nanmean(stn_pcp_std[mask_stn_t,:,:],axis=0)     
stn_tmean_std_mean = np.nanmean(stn_tmean_std[mask_stn_t,:,:],axis=0)
stn_trange_std_mean = np.nanmean(stn_trange_std[mask_stn_t,:,:],axis=0)

print(' -- masked values')
# masked values
stn_pcp_std_mean=np.where(mask_both==True,np.nan,stn_pcp_std_mean)
stn_tmean_std_mean=np.where(mask_both==True,np.nan,stn_tmean_std_mean)
stn_trange_std_mean=np.where(mask_both==True,np.nan,stn_trange_std_mean)

#======================================================================================================
# plot each sceanrio std vs. N15 std
# for k in range(len(test_folders)):
for k in range(len(test_folders)-2,len(test_folders)-1):
    test_folder = test_folders[k]
    print(test_folder)

    print('Read NLDAS regression results')
    output_basename = os.path.join(nldas_regr_dir, test_folder,'gmet_ens_bc_summary', 'ens_forc.2016.ensstd.nc')
    fig_title= test_folder

    print(' -- read spatial uncertainty')
    # read regression uncertainty    
    f=xr.open_dataset(output_basename)
    time = f['time'].values[:]
    time_regr = pd.to_datetime(time)

    nldas_pcp_std = f['pcp'].values[:]
    nldas_tmean_std = f['t_mean'].values[:]
    nldas_trange_std = f['t_range'].values[:]

    # define plot mask for nldas ensemble
    mask_ens_t = (time_regr>=plot_date_start_obj) & (time_regr<=plot_date_end_obj)

    print(' -- calculate temporal mean')
    # caluclate time series mean(ny,nx)
    nldas_pcp_std_mean = np.nanmean(nldas_pcp_std[mask_ens_t,:,:],axis=0)     
    nldas_tmean_std_mean = np.nanmean(nldas_tmean_std[mask_ens_t,:,:],axis=0)
    nldas_trange_std_mean = np.nanmean(nldas_trange_std[mask_ens_t,:,:],axis=0)

    print(' -- masked values')
    # masked values
    nldas_pcp_std_mean=np.where(mask_both==True,np.nan,nldas_pcp_std_mean)
    nldas_tmean_std_mean=np.where(mask_both==True,np.nan,nldas_tmean_std_mean)
    nldas_trange_std_mean=np.where(mask_both==True,np.nan,nldas_trange_std_mean)

    #======================================================================================================
    print('calcualte difference')
    nldas_pcp_std_diff = nldas_pcp_std_mean - stn_pcp_std_mean
    tmean_error_diff = nldas_tmean_std_mean - stn_tmean_std_mean
    trange_error_diff = nldas_trange_std_mean - stn_trange_std_mean

    #======================================================================================================
    print('calcualte vmin and vmax')

    vmin_nldas_pcp_std=np.nanmin([np.nanmin(nldas_pcp_std_mean),np.nanmin(stn_pcp_std_mean)])
    vmax_nldas_pcp_std=np.nanmin([np.nanmax(nldas_pcp_std_mean),np.nanmax(stn_pcp_std_mean)])
    lim=max([abs(np.nanmin(nldas_pcp_std_diff)),abs(np.nanmax(nldas_pcp_std_diff))])
    vmin_pcp_diff=(-1)*(lim)
    vmax_pcp_diff=lim

    vmin_tmean_error=np.nanmin([np.nanmin(nldas_tmean_std_mean),np.nanmin(stn_tmean_std_mean)])
    vmax_tmean_error=np.nanmin([np.nanmax(nldas_tmean_std_mean),np.nanmax(stn_tmean_std_mean)])
    lim=max([abs(np.nanmin(nldas_tmean_std_mean)),abs(np.nanmax(stn_tmean_std_mean))])
    vmin_tmean_diff=(-1)*(lim)
    vmax_tmean_diff=lim

    vmin_trange_error=np.nanmin([np.nanmin(nldas_trange_std_mean),np.nanmin(stn_trange_std_mean)])
    vmax_trange_error=np.nanmin([np.nanmax(nldas_trange_std_mean),np.nanmax(stn_trange_std_mean)])
    lim=max([abs(np.nanmin(nldas_trange_std_mean)),abs(np.nanmax(stn_trange_std_mean))])
    vmin_trange_diff=(-1)*(lim)
    vmax_trange_diff=lim

    #======================================================================================================
    print('Plot')
    # plot
    nrow = 3 # precip, tmean, trange
    ncol = 3 # stn std, nldas std, difference   
    fig, ax = plt.subplots(nrow, ncol, figsize=(7.08,7.08*0.70))
    # figure size (width, height) 
    #constrained_layout=True
    fig.subplots_adjust(left=0.1, bottom=0.1, right=1, top=0.9)#, wspace=None, hspace=None)
    ylabels=['Precip','Tmean','DTR']

    llcrnrlon = longitude[0,0]
    urcrnrlon = longitude[-1,-1]
    llcrnrlat = latitude[0,0]
    urcrnrlat = latitude[-1,-1]
    lat_0=0.5*(llcrnrlat+urcrnrlat)
    lon_0=0.5*(llcrnrlon+urcrnrlon)
    (ny,nx)=np.shape(longitude)

    for i in range(nrow):
        for j in range(ncol):

            # select data for each subplot
            # PCP (1st row)
            if i == 0:
                if j<(ncol-1):
                    cmap,vmin,vmax=plt.cm.terrain_r, vmin_nldas_pcp_std, vmax_nldas_pcp_std #plt.cm.jet_r
                    if j == 0:
                        data,title_str=stn_pcp_std_mean,'(a) N15 Std Dev'
                    elif j==1:
                        data,title_str=nldas_pcp_std_mean,'(b) NLDAS Std Dev'
                elif j==(ncol-1):
                    cmap,vmin,vmax=plt.cm.bwr, vmin_pcp_diff, vmax_pcp_diff
                    data,title_str=nldas_pcp_std_diff,'(c) NLDAS - N15'   
            elif i == 1:
                if j<(ncol-1):
                    cmap,vmin,vmax=plt.cm.jet, vmin_tmean_error, vmax_tmean_error 
                    if j == 0:
                        data=stn_tmean_std_mean
                    elif j==1:
                        data=nldas_tmean_std_mean
                elif j==(ncol-1):
                    cmap,vmin,vmax=plt.cm.bwr, vmin_tmean_diff, vmax_tmean_diff
                    data=tmean_error_diff
            elif i == 2:
                if j<(ncol-1):
                    cmap,vmin,vmax=plt.cm.jet, vmin_trange_error, vmax_trange_error 
                    if j == 0:
                        data=stn_trange_std_mean
                    elif j==1:
                        data=nldas_trange_std_mean
                elif j==(ncol-1):
                    cmap,vmin,vmax=plt.cm.bwr, vmin_trange_diff, vmax_trange_diff
                    data=trange_error_diff

            # plot Basemap
            m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax[i,j],lat_0,lon_0,ny,nx) # plot Basemap 

            # plot data
            im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap,vmin=vmin,vmax=vmax)
#             im = m.pcolormesh(longitude,latitude,data,shading='flat',latlon=True,cmap=cmap)

            # set colorbar
            if i == 0:
                cbar_label ='$(mm/day)$'
            elif i == 1:
                cbar_label = '($^\circ$C)'
            elif i == 2:
                cbar_label = '($^\circ$C)'

            cbar = fig.colorbar(im,ax=ax[i,j],pad=0.03,orientation="horizontal")  
            cbar.ax.tick_params(labelsize='xx-small',pad=0.05, length=2)             
            cbar.set_label(label=cbar_label, size='xx-small', rotation='horizontal', labelpad=-0.5)

            # set title
            if i == 0:
                ax[i,j].set_title(title_str, fontsize='xx-small', fontweight='semibold')
            # set ylabel
            if j == 0:
                ax[i,j].set_ylabel(ylabels[i], fontsize='xx-small', fontweight='semibold')

    # save plot
    fig.tight_layout(pad=0.2)
    output_filename = test_folder+'.png'
    fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value)
    plt.close(fig)

print('Done')


Read gridinfo mask
Read station ens std
 -- read spatial uncertainty
 -- calculate temporal mean
 -- masked values
08884grids
Read NLDAS regression results
 -- read spatial uncertainty


<ipython-input-5-8f4276476cac>:121: RuntimeWarning: Mean of empty slice
  stn_pcp_std_mean = np.nanmean(stn_pcp_std[mask_stn_t,:,:],axis=0)
<ipython-input-5-8f4276476cac>:122: RuntimeWarning: Mean of empty slice
  stn_tmean_std_mean = np.nanmean(stn_tmean_std[mask_stn_t,:,:],axis=0)
<ipython-input-5-8f4276476cac>:123: RuntimeWarning: Mean of empty slice
  stn_trange_std_mean = np.nanmean(stn_trange_std[mask_stn_t,:,:],axis=0)


 -- calculate temporal mean
 -- masked values
calcualte difference
calcualte vmin and vmax
Plot


<ipython-input-5-8f4276476cac>:157: RuntimeWarning: Mean of empty slice
  nldas_pcp_std_mean = np.nanmean(nldas_pcp_std[mask_ens_t,:,:],axis=0)
<ipython-input-5-8f4276476cac>:158: RuntimeWarning: Mean of empty slice
  nldas_tmean_std_mean = np.nanmean(nldas_tmean_std[mask_ens_t,:,:],axis=0)
<ipython-input-5-8f4276476cac>:159: RuntimeWarning: Mean of empty slice
  nldas_trange_std_mean = np.nanmean(nldas_trange_std[mask_ens_t,:,:],axis=0)


Done


In [2]:
time_regr,plot_date_start_obj,plot_date_end_obj

(DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
                '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
                '2016-01-09', '2016-01-10',
                ...
                '2016-12-22', '2016-12-23', '2016-12-24', '2016-12-25',
                '2016-12-26', '2016-12-27', '2016-12-28', '2016-12-29',
                '2016-12-30', '2016-12-31'],
               dtype='datetime64[ns]', length=366, freq=None),
 datetime.datetime(2016, 6, 1, 0, 0),
 datetime.datetime(2016, 6, 30, 0, 0))

In [43]:
f=xr.open_dataset(output_basename)
time = f['time'].values[:]
time_regr = pd.to_datetime(time)


In [4]:
nldas_pcp_std_mean

array([[       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       [       nan,        nan,        nan, ...,        nan,        nan,
               nan],
       ...,
       [0.8352764 , 0.839096  , 0.75159305, ..., 0.53248787, 0.52095646,
        0.5100946 ],
       [0.6849473 , 0.67363113, 0.7172282 , ..., 0.51569855, 0.49981996,
        0.49847978],
       [0.7168275 , 0.6997193 , 0.70697796, ..., 0.49234143, 0.49044114,
        0.47996023]], dtype=float32)